In [ ]:
!wget https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz
!tar -zxvf flower_photos.tgz
!rm -rf images
!mv flower_photos images
!rm images/LICENSE.txt
!rm -rf images/daisy
!rm -rf images/roses
!rm -rf images/tulips

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [ ]:
# 此例子是使用tensorflow框架结合MobileNet V2模型，在imagenet预训练参数下微调成一个图像分类器。
# 1、需要安装最新版的tensorflow 当前为pip install tensorflow==2.0.0-beta1
# 2、base_dir是放图片目录，此目录下还有许多子目录，每个子目录存放一个类别的图像
# 3、此例子在mybinder运行，只是为了演示，实际训练请修改BATCH_SIZE，EPOCHS或者其它代码

import tensorflow as tf
import os
import shutil
import numpy as np


base_dir="images"
IMAGE_SIZE = 224
BATCH_SIZE = 10 #64
EPOCHS=1 #30

#创建一个样本增强生成器，validation_split即为训练集与验证集比例是8:2
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2)

#创建训练集生成器，使用枚举“training”指定
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

#创建验证集生成器，使用枚举“validation”指定
val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')

#根据子文件夹名来生成标签
labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

#载入预训练模型MobileNet V2，去除顶部原有分类器用于做fine-tune,并指定基础模型的采用imagenet数据集进行预训练。
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

#先把基础模型所有层设为可训练（所有的层参与训练）
base_model.trainable = True

#所有层都训练就太慢了，所以从100层以后的参数可以训练比较快。
fine_tune_at = 100
#把基础模型的前100层都冻结不参与训练
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

#在基础模型上添加一个自己的简单分类器
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(len(train_generator.class_indices.keys()), activation='softmax')
])


#加上损失函数与优化器
model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(1e-5),
              metrics=['accuracy'])

#训练
history_fine = model.fit(train_generator, 
                         epochs=EPOCHS,
                         validation_data=val_generator)

#保存模型
saved_model_dir = 'save/fine_tuning'
if os.path.isdir(saved_model_dir):
    shutil.rmtree(saved_model_dir)
tf.saved_model.save(model, saved_model_dir)

#转换模型为tflite
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

#保存tflite模型文件
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

print("ok")

Found 1279 images belonging to 2 classes.
Found 318 images belonging to 2 classes.


W0716 03:31:35.657920 139989764822848 deprecation.py:323] From /srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


  5/128 [>.............................] - ETA: 14:03 - loss: 0.6702 - accuracy: 0.6200